In [1]:
dbutils.fs.mv("dbfs:/FileStore/tables/glove_6B_100d-15091.txt","file:/databricks/driver/glove.6B.100d.txt")

In [2]:
###################################### Prepare data for training Model #####################################

In [3]:
import pyspark
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text as txt
from numpy import asarray,array
from numpy import zeros

Lreviewdata_full = spark.read.parquet("/mnt/etl/parquet/labeled/Lreviewdata")
Lreviewdata = Lreviewdata_full.select("Text","label")
Lreviewdata.registerTempTable("DF")
Lreviewdata.cache()


# Training and testing split
train_0, test_0 = Lreviewdata.filter('label = 0.0').randomSplit([0.7, 0.3])
train_1, test_1 = Lreviewdata.filter('label = 1.0').randomSplit([0.7, 0.3])
test = test_0.union(test_1)
train = train_0.union(train_1)
traindataset = spark.createDataFrame(train.rdd.repartition(8))
testdataset = spark.createDataFrame(test.rdd.repartition(8))

sqlContext.clearCache()
traindataset.cache()
testdataset.cache()


l = traindataset.select('Text').rdd.map(lambda x: x[0]).collect()
m = testdataset.select('Text').rdd.map(lambda x: x[0]).collect()

tk = txt.Tokenizer(split= " ")   #tokenizes  
tk.fit_on_texts(l)
tk_test = txt.Tokenizer(split= " ")   #tokenizes  
tk_test.fit_on_texts(m)

#create train data set, both features and labels
x = tk.texts_to_sequences(l)      # converts the text to numbers
y = traindataset.select('label').rdd.map(lambda x: x[0]).collect()
#create test data set, both features and labels
x_test = tk_test.texts_to_sequences(m)      # converts the text to numbers
y_test = testdataset.select('label').rdd.map(lambda x: x[0]).collect()

# assign attributes to variables for consistency
max_features = 20000   # the more the better
max_length = 100  # cut texts after this number of words (arbitray)

# pad the train sequences so all arrays of same length
x = sequence.pad_sequences(x, maxlen = max_length, padding = 'post')  
# pad the test sequences so all arrays of same length
x_test = sequence.pad_sequences(x_test, maxlen = max_length, padding = 'post')  

vocab_size = len(tk.word_index) + 1

# load the whole embedding into memory
embeddings_index = dict()
f = open('/databricks/driver/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in tk.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector



Loaded 400000 word vectors.

In [4]:
######################################  Training with Dense layers ######################################

In [5]:
############################# Build Dense deep layer model ##############################

from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding

# define model
densemodel = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=100, trainable=False)
densemodel.add(e)
densemodel.add(Flatten())
densemodel.add(Dense(1, activation='sigmoid'))
# compile the model
densemodel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(densemodel.summary())
# fit the model
densemodel.fit(x, y, validation_data=(x_test, y_test),epochs=2, verbose=0)

# Final evaluation of the model
scores = densemodel.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

WARNING:tensorflow:From /databricks/python/lib/python3.5/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
embedding_1 (Embedding) (None, 100, 100) 17287600 
_________________________________________________________________
flatten_1 (Flatten) (None, 10000) 0 
_________________________________________________________________
dense_1 (Dense) (None, 1) 10001 
=================================================================
Total params: 17,297,601
Trainable params: 10,001
Non-trainable params: 17,287,600
_________________________________________________________________
None
WARNING:tensorflow:From /databricks/python/lib/python3.5/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
Accuracy: 84.46%

In [6]:
dbutils.fs.mkdirs("/mnt/TrainedModels/")

Out[ 9 ]: True

In [7]:
densemodel.save("/databricks/driver/Fakereviews_Deeplearning_dense_model.h5")
dbutils.fs.mv("file:///databricks/driver/Fakereviews_Deeplearning_dense_model.h5","dbfs:/mnt/TrainedModels/Fakereviews_Deeplearning_dense_model.h5")

Out[ 15 ]: True

In [8]:
###################################### Training with CNN model ######################################

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding
from keras.layers.convolutional import Convolution1D, MaxPooling1D

# define model
CNNmodel = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=100, trainable=False)
CNNmodel.add(e)
CNNmodel.add(Convolution1D(nb_filter=32, filter_length=10, border_mode='same', activation='relu'))
CNNmodel.add(MaxPooling1D(pool_length=4))       # pooling (max) after convoluting
CNNmodel.add(Flatten())
CNNmodel.add(Dense(500, activation='relu'))     # relu and sigmoid
CNNmodel.add(Dense(1, activation='sigmoid'))
CNNmodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(CNNmodel.summary())

# Fit the model
CNNmodel.fit(x, y, validation_data=(x_test, y_test), nb_epoch=3, batch_size=20, verbose=1)   # batch size powers of two
# Final evaluation of the model
scores = CNNmodel.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

/local_disk0/tmp/1555067779100-0/PythonShell.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(kernel_size=10, padding="same", activation="relu", filters=32)`
 import six.moves.queue
/local_disk0/tmp/1555067779100-0/PythonShell.py:10: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=4)`
 import resource
_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
embedding_3 (Embedding) (None, 100, 100) 17287600 
_________________________________________________________________
conv1d_1 (Conv1D) (None, 100, 32) 32032 
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 25, 32) 0 
_________________________________________________________________
flatten_2 (Flatten) (None, 800) 0 
_________________________________________________________________
dense_2 (Dense) (None, 500) 400500 
_________________________________________________________________
dense_3 (Dense) (None, 1) 501 
=================================================================
Total params: 17,720,633
Trainable params: 433,033
Non-trainable params: 17,287,600
_________________________________________________________________
None
/local_disk0/tmp/1555067779100-0/PythonShell.py:18: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
 from six.moves import range
Train on 430677 samples, validate on 183775 samples
Epoch 1/3

 20/430677 [..............................] - ETA: 4:07:25 - loss: 0.8442 - acc: 0.1500
 120/430677 [..............................] - ETA: 44:35 - loss: 0.8624 - acc: 0.7667 
 220/430677 [..............................] - ETA: 26:03 - loss: 0.8562 - acc: 0.7818
 340/430677 [..............................] - ETA: 18:04 - loss: 0.7570 - acc: 0.8118
 440/430677 [..............................] - ETA: 14:47 - loss: 0.7124 - acc: 0.8182
 560/430677 [..............................] - ETA: 12:21 - loss: 0.6569 - acc: 0.8232
 680/430677 [..............................] - ETA: 10:46 - loss: 0.6399 - acc: 0.8191
 800/430677 [..............................] - ETA: 9:40 - loss: 0.6052 - acc: 0.8263 
 920/430677 [..............................] - ETA: 8:51 - loss: 0.5777 - acc: 0.8337
 1040/430677 [..............................] - ETA: 8:11 - loss: 0.5657 - acc: 0.8337
 1160/430677 [..............................] - ETA: 7:42 - loss: 0.5550 - acc: 0.8336
 1280/430677 [..............................] - ETA: 7:18 - loss: 0.5503 - acc: 0.8305
 1400/430677 [..............................] - ETA: 6:57 - loss: 0.5398 - acc: 0.8329
 1520/430677 [..............................] - ETA: 6:40 - loss: 0.5300 - acc: 0.8355
 1640/430677 [..............................] - ETA: 6:26 - loss: 0.5173 - acc: 0.8390
 1760/430677 [..............................] - ETA: 6:13 - loss: 0.5083 - acc: 0.8415
 1880/430677 [..............................] - ETA: 6:02 - loss: 0.5024 - acc: 0.8431
 2000/430677 [..............................] - ETA: 5:52 - loss: 0.4980 - acc: 0.8435
 2120/430677 [..............................] - ETA: 5:43 - loss: 0.4915 - acc: 0.8458
 2240/430677 [..............................] - ETA: 5:36 - loss: 0.4867 - acc: 0.8469
 2360/430677 [..............................] - ETA: 5:29 - loss: 0.4839 - acc: 0.8475
 2480/430677 [..............................] - ETA: 5:23 - loss: 0.4785 - acc: 0.8492
 2600/430677 [..............................] - ETA: 5:18 - loss: 0.4773 - acc: 0.8485
 2720/430677 [..............................] - ETA: 5:13 - loss: 0.4781 - acc: 0.8471
 2840/430677 [..............................] - ETA: 5:08 - loss: 0.4755 - acc: 0.8486
 2960/430677 [..............................] - ETA: 5:03 - loss: 0.4747 - acc: 0.8483
 3080/430677 [..............................] - ETA: 4:59 - loss: 0.4709 - acc: 0.8497
 3200/430677 [..............................] - ETA: 4:56 - loss: 0.4686 - acc: 0.8500
 3320/430677 [.....

In [10]:
CNNmodel.save("/databricks/driver/Fakereviews_Deeplearning_CNN_model.h5")
dbutils.fs.mv("file:///databricks/driver/Fakereviews_Deeplearning_CNN_model.h5","dbfs:/mnt/TrainedModels/Fakereviews_Deeplearning_CNN_model.h5")

Out[ 23 ]: True

In [11]:
###################################### Training with RNN LSTM Model ######################################

In [12]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,LSTM, Dense, Dropout, Activation
from keras.layers import Flatten
from keras.layers import Embedding

# define model
LSTMmodel = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=100, trainable=False)
LSTMmodel.add(e)
LSTMmodel.add(LSTM(160, dropout=0.15, recurrent_dropout=0.15))
LSTMmodel.add(Dense(units=130, kernel_initializer='uniform', activation='relu'))
#model.add(Dense(units=80, kernel_initializer='uniform', activation='relu'))
LSTMmodel.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
LSTMmodel.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(LSTMmodel.summary())

# Fit the model
LSTMmodel.fit(x, y, validation_data=(x_test, y_test), nb_epoch=3, batch_size=50, verbose=1)   # batch size powers of two
# Final evaluation of the model
scores = LSTMmodel.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

WARNING:tensorflow:From /databricks/python/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
embedding_4 (Embedding) (None, 100, 100) 17287600 
_________________________________________________________________
lstm_1 (LSTM) (None, 160) 167040 
_________________________________________________________________
dense_4 (Dense) (None, 130) 20930 
_________________________________________________________________
dense_5 (Dense) (None, 1) 131 
=================================================================
Total params: 17,475,701
Trainable params: 188,101
Non-trainable params: 17,287,600
_________________________________________________________________
None
/local_disk0/tmp/1555067779100-0/PythonShell.py:19: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
 from tempfile import NamedTemporaryFile
Train on 430677 samples, validate on 183775 samples
Epoch 1/3

 50/430677 [..............................] - ETA: 3:09:47 - loss: 0.6938 - acc: 0.2800
 100/430677 [..............................] - ETA: 1:43:29 - loss: 0.6872 - acc: 0.5600
 150/430677 [..............................] - ETA: 1:14:51 - loss: 0.6641 - acc: 0.6667
 200/430677 [..............................] - ETA: 1:00:27 - loss: 0.6415 - acc: 0.7150
 250/430677 [..............................] - ETA: 51:50 - loss: 0.5913 - acc: 0.7480 
 300/430677 [..............................] - ETA: 46:11 - loss: 0.5468 - acc: 0.7733
 350/430677 [..............................] - ETA: 42:06 - loss: 0.5218 - acc: 0.7886
 400/430677 [..............................] - ETA: 38:57 - loss: 0.4887 - acc: 0.8075
 450/430677 [..............................] - ETA: 36:32 - loss: 0.4717 - acc: 0.8178
 500/430677 [..............................] - ETA: 34:35 - loss: 0.4912 - acc: 0.8100
 550/430677 [..............................] - ETA: 33:00 - loss: 0.4727 - acc: 0.8236
 600/430677 [..............................] - ETA: 31:45 - loss: 0.4641 - acc: 0.8283
 650/430677 [..............................] - ETA: 30:42 - loss: 0.4592 - acc: 0.8308
 700/430677 [..............................] - ETA: 29:46 - loss: 0.4635 - acc: 0.8286
 750/430677 [..............................] - ETA: 28:55 - loss: 0.4641 - acc: 0.8280
 800/430677 [..............................] - ETA: 28:10 - loss: 0.4645 - acc: 0.8275
 850/430677 [..............................] - ETA: 27:33 - loss: 0.4559 - acc: 0.8329
 900/430677 [..............................] - ETA: 26:58 - loss: 0.4469 - acc: 0.8378
 950/430677 [..............................] - ETA: 26:27 - loss: 0.4451 - acc: 0.8389
 1000/430677 [..............................] - ETA: 26:00 - loss: 0.4393 - acc: 0.8420
 1050/430677 [..............................] - ETA: 25:35 - loss: 0.4338 - acc: 0.8448
 1100/430677 [..............................] - ETA: 25:11 - loss: 0.4367 - acc: 0.8436
 1150/430677 [..............................] - ETA: 24:50 - loss: 0.4338 - acc: 0.8452
 1200/430677 [..............................] - ETA: 24:30 - loss: 0.4311 - acc: 0.8467
 1250/430677 [..............................] - ETA: 24:13 - loss: 0.4285 - acc: 0.8480
 1300/430677 [..............................] - ETA: 23:57 - loss: 0.4292 - acc: 0.8477
 1350/430677 [..............................] - ETA: 23:42 - loss: 0.4283 - acc: 0.8481
 1400/430677 [..............................] - ETA: 23:27 - loss: 0.4250 - acc: 0.8500
 1450/430677 [..............................] - ETA: 23:14 - loss: 0.4257 - acc: 0.8497
 1500/430677 [..............................] - ETA: 23:01 - loss: 0.4238 - acc: 0.8507
 1550/430677 [..............................] - ETA: 22

In [13]:
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 85.94%

In [14]:
LSTMmodel.save("/databricks/driver/Fakereviews_Deeplearning_LSTM_model.h5")
dbutils.fs.mv("file:///databricks/driver/Fakereviews_Deeplearning_LSTM_model.h5","dbfs:/mnt/TrainedModels/Fakereviews_Deeplearning_LSTM_model.h5")

Out[ 25 ]: True

In [15]:
%fs ls /mnt/TrainedModels/

path,name,size
dbfs:/mnt/TrainedModels/Fakereviews_Deeplearning_CNN_model.h5,Fakereviews_Deeplearning_CNN_model.h5,74379536
dbfs:/mnt/TrainedModels/Fakereviews_Deeplearning_LSTM_model.h5,Fakereviews_Deeplearning_LSTM_model.h5,70679552
dbfs:/mnt/TrainedModels/Fakereviews_Deeplearning_dense_model.h5,Fakereviews_Deeplearning_dense_model.h5,69292568
